In [1]:
# this is the dataset to use
# https://github.com/ofrendo/WebDataIntegration/blob/7db877abadd2be94d5373f5f47c8ccd1d179bea6/data/goldstandard/forbes_freebase_goldstandard_train.csv

In [2]:
!wget https://raw.githubusercontent.com/ofrendo/WebDataIntegration/7db877abadd2be94d5373f5f47c8ccd1d179bea6/data/goldstandard/forbes_freebase_goldstandard_train.csv

--2020-10-11 23:37:06--  https://raw.githubusercontent.com/ofrendo/WebDataIntegration/7db877abadd2be94d5373f5f47c8ccd1d179bea6/data/goldstandard/forbes_freebase_goldstandard_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7324 (7.2K) [text/plain]
Saving to: ‘forbes_freebase_goldstandard_train.csv’

forbes_freebase_gol 100%[===================>]   7.15K  --.-KB/s    in 0.009s  

2020-10-11 23:37:07 (782 KB/s) - ‘forbes_freebase_goldstandard_train.csv’ saved [7324/7324]



In [22]:
import pandas as pd

data = pd.read_csv('forbes_freebase_goldstandard_train.csv', names=['string1', 'string2', 'matched'])

loading training data

In [23]:
!wget https://raw.githubusercontent.com/ofrendo/WebDataIntegration/7db877abadd2be94d5373f5f47c8ccd1d179bea6/data/goldstandard/forbes_freebase_goldstandard_test.csv

--2020-10-11 23:54:11--  https://raw.githubusercontent.com/ofrendo/WebDataIntegration/7db877abadd2be94d5373f5f47c8ccd1d179bea6/data/goldstandard/forbes_freebase_goldstandard_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 976 [text/plain]
Saving to: ‘forbes_freebase_goldstandard_test.csv.1’

forbes_freebase_gol 100%[===================>]     976  --.-KB/s    in 0s      

2020-10-11 23:54:12 (20.2 MB/s) - ‘forbes_freebase_goldstandard_test.csv.1’ saved [976/976]



In [24]:
data.head()

,string1,string2,matched
0,General Electric,General Electric,True
1,Wells Fargo,Wells Fargo,True
2,Bank of China,Industrial and Commercial Bank of China (Asia),True
3,PetroChina,PetroChina,True
4,Apple,Apple Inc.,True


In [6]:
len(data)

212

In [7]:
!pip install python-Levenshtein

     |████████████████████████████████| 48 kB 2.0 MB/s eta 0:00:011
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-macosx_10_7_x86_64.whl size=80048 sha256=63813405d0de8ace271d0d15f41461ddea38df8cf475b3390e9bed0ee68ba08f
  Stored in directory: /Users/ben/Library/Caches/pip/wheels/79/c3/a1/cbdd8b154234b3e571d121b65be7d53354cc77e223e8f271c8
Successfully built python-Levenshtein


In [8]:
import joblib
import os
from multiprocessing import Pool
import re
from difflib import SequenceMatcher  # for longest common substring
from functools import partial
from operator import itemgetter
import Levenshtein  # levenstein/edit distance; docs here: https://rawgit.com/ztane/python-Levenshtein/master/docs/Levenshtein.html

In [9]:
def clean_string(string):
    '''We will use this functions to remove special characters etc before 
    any string distance calculation.
    '''
    return ''.join(map(lambda x: x.lower() if str.isalnum(x) else ' ', string)).strip()

I think it's useful to make a distinction here
- distance functions between two strings
- string featurization

The string distances we can use to establish a baseline performance. But with small changes we can make them string featurization functions and use them in classifier functions in a machine learning approach. However, let us get first to string distances.


In [34]:
def levenstein_distance(s1_, s2_):
    s1, s2 = clean_string(s1_), clean_string(s2_)
    len_s1, len_s2 = len(s1), len(s2)
    return Levenshtein.distance(
        s1, s2
    ) / max([len_s1, len_s2])

def jaro_winkler_distance(s1_, s2_):
    s1, s2 = clean_string(s1_), clean_string(s2_)
    return Levenshtein.jaro_winkler(s1, s2)

def common_substring_distance(s1_, s2_):
    s1, s2 = clean_string(s1_), clean_string(s2_)
    len_s1, len_s2 = len(s1), len(s2)
    match = SequenceMatcher(
        None, s1, s2
    ).find_longest_match(0, len_s1, 0, len_s2)
    len_s1, len_s2 = len(s1), len(s2)
    norm = max([len_s1, len_s2])
    return 1 - min([1, match.size / norm])


In [35]:
import numpy as np
from sklearn.metrics import roc_auc_score

dists = np.zeros(shape=(len(data), 3))
for algo_i, algo in enumerate(
    [levenstein_distance, jaro_winkler_distance, common_substring_distance]
):
    for i, string_pair in data.iterrows():
        dists[i, algo_i] = algo(string_pair['string1'], string_pair['string2'])
        
    print('AUC for {}: {}'.format(
        algo.__name__, 
        roc_auc_score(data['matched'].astype(float), 1 - dists[:, algo_i])
    ))
#AUC for levenstein_distance: 0.9508904955034385
#AUC for jaro_winkler_distance: 0.9470992770234525
#AUC for common_substring_distance: 0.9560042320578381

AUC for levenstein_distance: 0.9508904955034385
AUC for jaro_winkler_distance: 0.052900722976547354
AUC for common_substring_distance: 0.9560042320578381


In [36]:
# we can use these string functions for fuzzy string match
# some matches are not very good, so we should make sure, we have thresholds.

pool = Pool(50)

def fuzzy_string_search(
    s1, string_list,
     string_compare,
      threshold=lambda sim: sim>0.7
    ):
    '''Search through a list of strings using a string_comparison function
    in order to find the best match.

    Parameters:
    - s1: string to search for
    - string_list: list of strings
    - string_compare: string comparison function to return a similarity or a 
      distance.
    - threshold: cut-off function to decide if returning the best match or
      nothing at all. This threshold function has to take into account if we
      are using a string similarity or a string distance.

    Return the best matching string if threshold reached.
    Otherwise return None.

    Example:
    >> company_list = [
        'Blackrock',
        'Credit Suisse',
        'Goldman Sachs',
        'Bank of America/Meryll Lynch',
        'Morgan Stanley',
        'LEK',
        'JP Morgan',
        'Nomura',
        'BNP Paribas',
        'WPP',
        'Rothschild',
        'Allianz',
    ]
    >> fuzzy_string_search(
      'SAP',
      company_list,
      jaro_winkler_distance,
      threshold=lambda dist: dist<0.7
    )
    '''
    string_match = partial(string_compare, s2_=s1)
    comparisons = pool.map(string_match, string_list)
    index, element = max(enumerate(comparisons), key=itemgetter(1))
    #print(f"match {string_list[index]} with score {element}")
    if threshold(element):
        return string_list[index]
    else:
        return None

In [37]:
company_list = [
        'Blackrock',
        'Credit Suisse',
        'Goldman Sachs',
        'Bank of America/Meryll Lynch',
        'Morgan Stanley',
        'LEK',
        'JP Morgan',
        'Nomura',
        'BNP Paribas',
        'WPP',
        'Rothschild',
        'Allianz',
    ]
fuzzy_string_search(
      'SAP',
      company_list,
      jaro_winkler_distance,
      threshold=lambda dist: dist<0.7
    )


'WPP'

In [65]:
# We can also use featurizations of strings, for example using sklearn
# inbuilt functionality such as CountVectorizers or TfidfVectorizer.


from sklearn.feature_extraction.text import CountVectorizer
# the CountVectorizer counts the occurences of features. These features
# can be composed of characters or words; we are interested in character-
# based features. We clean the strings as before and we take ngrams.
# Also try TfidfVectorizer for a baseline performance

# We can also use featurizations of strings, for example using sklearn
# inbuilt functionality such as CountVectorizers or TfidfVectorizer.


from sklearn.feature_extraction.text import CountVectorizer
# the CountVectorizer counts the occurences of features. These features
# can be composed of characters or words; we are interested in character-
# based features. We clean the strings as before and we take ngrams.
# Also try TfidfVectorizer for a baseline performance
from sklearn.feature_extraction.text import CountVectorizer

# We clean the strings as before and we take ngrams.
ngram_featurizer = CountVectorizer(
    min_df=1,
    analyzer='char',
    ngram_range=(1, 1), # this is the range of ngrams that are to be extracted!
    preprocessor=clean_string
)
company_space = ngram_featurizer.fit_transform(
    np.concatenate(
        [data['string1'], data['string2']],
        axis=0
    )
)

In [66]:
company_space

<426x41 sparse matrix of type '<class 'numpy.int64'>'
	with 4003 stored elements in Compressed Sparse Row format>

In [67]:
string1cv = ngram_featurizer.transform(data['string1'])
string2cv = ngram_featurizer.transform(data['string2'])

def norm(string1cv):
    return string1cv / string1cv.sum(axis=1)

similarities = 1 - np.sum(np.abs(norm(string1cv) - norm(string2cv)), axis=1) / 2
roc_auc_score(data['matched'].astype(float), similarities)
#0.9298183741844471


0.9298183741844471

In [68]:
# Some alternative way to featurize strings. These can be used in similar ways 
# to the CountVectorizer really. Apply these as a preprocessor to a classifier
# and check the performance in distinguishing between match and no-match.

def editops_featurizer(s1_, s2_):
    '''Counts the replace, insert and delete operations between two strings
    and normalizes these by maximum string length.

    This featurization could be interesting to find out which operation is
    most useful.
    '''
    s1, s2 = clean_string(s1_), clean_string(s2_)
    len_s1, len_s2 = len(s1), len(s2)
    ops = Levenshtein.editops(
        s1, s2
    )
    index_dict = {'insert': 0, 'replace': 1, 'delete': 2}
    features = np.zeros((3))
    for op in edit_ops:
        features[index_dict[op[0]]] += 1
    features / max([len_s1, len_s2])  
    return features

def common_substring_featurizer(s1_, s2_):
    '''Here we extract 1. the normalized length of the common substring
    and whether the common substring matches 2. the beginning or
    3. the end of a word.
    '''
    s1, s2 = clean_string(s1_), clean_string(s2_)
    len_s1, len_s2 = len(s1), len(s2)
    longer_string = s1 if len_s1 > len_s2 else s2
    norm = max([len_s1, len_s2])    
    match = SequenceMatcher(None, s1, s2).find_longest_match(0, len_s1, 0, len_s2)
    substring = s1[match.a: match.a + match.size]
    
    m1 = re.search(
        '(?:^|\s|[a-z])' + substring,
        longer_string
    )
    m2 = re.search(
        substring + '(?:[a-z]|\s|$)',
        longer_string
    )
    return min([1, match.size / norm]), m1 is not None, m2 is not None

In [69]:
# siamese network dimensionality reduction
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Lambda, Input
import tensorflow as tf
from tensorflow.keras import backend as K



def create_string_featurization_model(feature_dimensionality, output_dim=50):
    '''
    Use for string featurization in combination with siamese models.
    Just a non-linear projection as a way of reducing the feature dimensionality
    in a meaningful way.

    Parameters:
        feature_dimensionality - number of features coming from the vectorizer
          or string featurization function
        output_dim - dimensions of the embedding/projection that we are trying
          to create
    '''
    preprocessing_model = Sequential()
    preprocessing_model.add(
        Dense(output_dim, activation='selu', input_dim=feature_dimensionality)
    )
    preprocessing_model.summary()
    return preprocessing_model

def create_siamese_model(preprocessing_models, #initial_bias =
                          input_shapes=(10,)):
    def euclidean_distance(vects):
        x, y = vects
        x = K.l2_normalize(x, axis=-1)
        y = K.l2_normalize(y, axis=-1)
        sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
        return K.sqrt(K.maximum(sum_square, K.epsilon()))
    
    if not isinstance(preprocessing_models, (list, tuple)):
        raise ValueError('preprocessing models needs to be a list or tuple of models')

    print('{} models to be trained against each other'.format(len(preprocessing_models)))
    if not isinstance(input_shapes, list):
        input_shapes = [input_shapes] * len(preprocessing_models)
    
    inputs = []
    intermediate_layers = []
    for preprocessing_model, input_shape in zip(preprocessing_models, input_shapes):
        inputs.append(Input(shape=input_shape))
        intermediate_layers.append(preprocessing_model(inputs[-1]))

    layer_diffs = []
    for i in range(len(intermediate_layers)-1):        
        layer_diffs.append(
            Lambda(euclidean_distance)([intermediate_layers[i], intermediate_layers[i+1]])
        )    
    siamese_model = Model(inputs=inputs, outputs=layer_diffs)
    siamese_model.summary()
    return siamese_model

def compile_model(model):
    model.compile(
        optimizer='rmsprop',
        loss='mse',
        metrics=[
            #'accuracy',
            #tf.keras.metrics.FalseNegatives(name='fn'), 
            #tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall'),
            #tf.keras.metrics.AUC(name='auc'),
        ]
    )

# use like this:
feature_dims = len(ngram_featurizer.get_feature_names())
string_featurization_model = create_string_featurization_model(feature_dims, output_dim=50)

siamese_model = create_siamese_model(
    preprocessing_models=[string_featurization_model, string_featurization_model],
    input_shapes=[(feature_dims,), (feature_dims,)],
)
compile_model(siamese_model)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 50)                2100      
Total params: 2,100
Trainable params: 2,100
Non-trainable params: 0
_________________________________________________________________
2 models to be trained against each other
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 41)]         0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 41)]         0                                            
____________________________________________________________________________________________

In [70]:
import numpy as np
from sklearn.metrics import roc_auc_score

string1cv = ngram_featurizer.transform(data['string1'])
string2cv = ngram_featurizer.transform(data['string2'])

def norm(string1cv):
    return string1cv / string1cv.sum(axis=1)

similarities = 1 - np.sum(np.abs(norm(string1cv) - norm(string2cv)), axis=1) / 2
roc_auc_score(data['matched'].astype(float), similarities)
#0.9298183741844471

0.9298183741844471

In [71]:
siamese_model.fit( 
    [string1cv, string2cv],
    1 - data['matched'].astype(float),
    epochs=1000
)

Train on 213 samples
Epoch 1/1000
213/213 [==============================] - 0s 2ms/sample - loss: 0.1020 - recall: 0.9439
Epoch 2/1000
213/213 [==============================] - 0s 178us/sample - loss: 0.0950 - recall: 0.9439
Epoch 3/1000
213/213 [==============================] - 0s 316us/sample - loss: 0.0914 - recall: 0.9439
Epoch 4/1000
213/213 [==============================] - 0s 122us/sample - loss: 0.0882 - recall: 0.9439
Epoch 5/1000
213/213 [==============================] - 0s 161us/sample - loss: 0.0856 - recall: 0.9439
Epoch 6/1000
213/213 [==============================] - 0s 205us/sample - loss: 0.0827 - recall: 0.9533
Epoch 7/1000
213/213 [==============================] - 0s 150us/sample - loss: 0.0802 - recall: 0.9533
Epoch 8/1000
213/213 [==============================] - 0s 245us/sample - loss: 0.0778 - recall: 0.9533
Epoch 9/1000
213/213 [==============================] - 0s 196us/sample - loss: 0.0756 - recall: 0.9533
Epoch 10/1000
213/213 [======================

213/213 [==============================] - 0s 147us/sample - loss: 0.0301 - recall: 0.9720
Epoch 79/1000
213/213 [==============================] - 0s 178us/sample - loss: 0.0299 - recall: 0.9720
Epoch 80/1000
213/213 [==============================] - 0s 190us/sample - loss: 0.0297 - recall: 0.9720
Epoch 81/1000
213/213 [==============================] - 0s 203us/sample - loss: 0.0295 - recall: 0.9720
Epoch 82/1000
213/213 [==============================] - 0s 150us/sample - loss: 0.0294 - recall: 0.9720
Epoch 83/1000
213/213 [==============================] - 0s 147us/sample - loss: 0.0292 - recall: 0.9720
Epoch 84/1000
213/213 [==============================] - 0s 115us/sample - loss: 0.0292 - recall: 0.9720
Epoch 85/1000
213/213 [==============================] - 0s 124us/sample - loss: 0.0290 - recall: 0.9720
Epoch 86/1000
213/213 [==============================] - 0s 143us/sample - loss: 0.0290 - recall: 0.9720
Epoch 87/1000
213/213 [==============================] - 0s 221us/sam

213/213 [==============================] - 0s 199us/sample - loss: 0.0232 - recall: 0.9720
Epoch 156/1000
213/213 [==============================] - 0s 306us/sample - loss: 0.0232 - recall: 0.9720
Epoch 157/1000
213/213 [==============================] - 0s 490us/sample - loss: 0.0230 - recall: 0.9720
Epoch 158/1000
213/213 [==============================] - 0s 173us/sample - loss: 0.0230 - recall: 0.9720
Epoch 159/1000
213/213 [==============================] - ETA: 0s - loss: 0.0054 - recall: 1.00 - 0s 119us/sample - loss: 0.0229 - recall: 0.9720
Epoch 160/1000
213/213 [==============================] - 0s 136us/sample - loss: 0.0229 - recall: 0.9720
Epoch 161/1000
213/213 [==============================] - 0s 188us/sample - loss: 0.0229 - recall: 0.9720
Epoch 162/1000
213/213 [==============================] - 0s 178us/sample - loss: 0.0228 - recall: 0.9720
Epoch 163/1000
213/213 [==============================] - 0s 99us/sample - loss: 0.0228 - recall: 0.9720
Epoch 164/1000
213/213

213/213 [==============================] - 0s 113us/sample - loss: 0.0205 - recall: 0.9720
Epoch 233/1000
213/213 [==============================] - 0s 157us/sample - loss: 0.0205 - recall: 0.9720
Epoch 234/1000
213/213 [==============================] - 0s 179us/sample - loss: 0.0205 - recall: 0.9720
Epoch 235/1000
213/213 [==============================] - 0s 144us/sample - loss: 0.0204 - recall: 0.9720
Epoch 236/1000
213/213 [==============================] - 0s 123us/sample - loss: 0.0204 - recall: 0.9720
Epoch 237/1000
213/213 [==============================] - 0s 108us/sample - loss: 0.0204 - recall: 0.9720
Epoch 238/1000
213/213 [==============================] - 0s 106us/sample - loss: 0.0204 - recall: 0.9720
Epoch 239/1000
213/213 [==============================] - 0s 118us/sample - loss: 0.0204 - recall: 0.9720
Epoch 240/1000
213/213 [==============================] - 0s 117us/sample - loss: 0.0203 - recall: 0.9720
Epoch 241/1000
213/213 [==============================] - 0s 

213/213 [==============================] - 0s 199us/sample - loss: 0.0191 - recall: 0.9720
Epoch 310/1000
213/213 [==============================] - 0s 158us/sample - loss: 0.0191 - recall: 0.9720
Epoch 311/1000
213/213 [==============================] - 0s 178us/sample - loss: 0.0192 - recall: 0.9720
Epoch 312/1000
213/213 [==============================] - 0s 163us/sample - loss: 0.0192 - recall: 0.9720
Epoch 313/1000
213/213 [==============================] - 0s 158us/sample - loss: 0.0191 - recall: 0.9720
Epoch 314/1000
213/213 [==============================] - 0s 160us/sample - loss: 0.0191 - recall: 0.9720
Epoch 315/1000
213/213 [==============================] - 0s 174us/sample - loss: 0.0191 - recall: 0.9720
Epoch 316/1000
213/213 [==============================] - 0s 140us/sample - loss: 0.0191 - recall: 0.9720
Epoch 317/1000
213/213 [==============================] - 0s 132us/sample - loss: 0.0191 - recall: 0.9720
Epoch 318/1000
213/213 [==============================] - 0s 

...

213/213 [==============================] - 0s 175us/sample - loss: 0.0168 - recall: 0.9720
Epoch 618/1000
213/213 [==============================] - 0s 167us/sample - loss: 0.0168 - recall: 0.9720
Epoch 619/1000
213/213 [==============================] - 0s 168us/sample - loss: 0.0168 - recall: 0.9720
Epoch 620/1000
213/213 [==============================] - 0s 145us/sample - loss: 0.0168 - recall: 0.9720
Epoch 621/1000
213/213 [==============================] - 0s 153us/sample - loss: 0.0167 - recall: 0.9720
Epoch 622/1000
213/213 [==============================] - 0s 162us/sample - loss: 0.0167 - recall: 0.9720
Epoch 623/1000
213/213 [==============================] - 0s 122us/sample - loss: 0.0167 - recall: 0.9720
Epoch 624/1000
213/213 [==============================] - 0s 234us/sample - loss: 0.0168 - recall: 0.9720
Epoch 625/1000
213/213 [==============================] - 0s 398us/sample - loss: 0.0167 - recall: 0.9720
Epoch 626/1000
213/213 [==============================] - 0s 

213/213 [==============================] - 0s 142us/sample - loss: 0.0164 - recall: 0.9720
Epoch 695/1000
213/213 [==============================] - 0s 164us/sample - loss: 0.0164 - recall: 0.9720
Epoch 696/1000
213/213 [==============================] - 0s 103us/sample - loss: 0.0164 - recall: 0.9720
Epoch 697/1000
213/213 [==============================] - 0s 151us/sample - loss: 0.0164 - recall: 0.9720
Epoch 698/1000
213/213 [==============================] - 0s 112us/sample - loss: 0.0164 - recall: 0.9720
Epoch 699/1000
213/213 [==============================] - 0s 116us/sample - loss: 0.0164 - recall: 0.9720
Epoch 700/1000
213/213 [==============================] - 0s 135us/sample - loss: 0.0164 - recall: 0.9720
Epoch 701/1000
213/213 [==============================] - 0s 139us/sample - loss: 0.0164 - recall: 0.9720
Epoch 702/1000
213/213 [==============================] - 0s 161us/sample - loss: 0.0164 - recall: 0.9720
Epoch 703/1000
213/213 [==============================] - 0s 

213/213 [==============================] - 0s 97us/sample - loss: 0.0161 - recall: 0.9720
Epoch 772/1000
213/213 [==============================] - 0s 138us/sample - loss: 0.0161 - recall: 0.9720
Epoch 773/1000
213/213 [==============================] - 0s 164us/sample - loss: 0.0161 - recall: 0.9720
Epoch 774/1000
213/213 [==============================] - 0s 140us/sample - loss: 0.0161 - recall: 0.9720
Epoch 775/1000
213/213 [==============================] - 0s 185us/sample - loss: 0.0161 - recall: 0.9720
Epoch 776/1000
213/213 [==============================] - 0s 95us/sample - loss: 0.0161 - recall: 0.9720
Epoch 777/1000
213/213 [==============================] - 0s 123us/sample - loss: 0.0161 - recall: 0.9720
Epoch 778/1000
213/213 [==============================] - 0s 136us/sample - loss: 0.0161 - recall: 0.9720
Epoch 779/1000
213/213 [==============================] - 0s 150us/sample - loss: 0.0161 - recall: 0.9720
Epoch 780/1000
213/213 [==============================] - 0s 13

213/213 [==============================] - 0s 153us/sample - loss: 0.0159 - recall: 0.9720
Epoch 849/1000
213/213 [==============================] - 0s 208us/sample - loss: 0.0159 - recall: 0.9720
Epoch 850/1000
213/213 [==============================] - 0s 174us/sample - loss: 0.0159 - recall: 0.9720
Epoch 851/1000
213/213 [==============================] - 0s 212us/sample - loss: 0.0159 - recall: 0.9720
Epoch 852/1000
213/213 [==============================] - 0s 425us/sample - loss: 0.0158 - recall: 0.9720
Epoch 853/1000
213/213 [==============================] - 0s 168us/sample - loss: 0.0158 - recall: 0.9720
Epoch 854/1000
213/213 [==============================] - 0s 119us/sample - loss: 0.0158 - recall: 0.9720
Epoch 855/1000
213/213 [==============================] - 0s 147us/sample - loss: 0.0159 - recall: 0.9720
Epoch 856/1000
213/213 [==============================] - 0s 193us/sample - loss: 0.0158 - recall: 0.9720
Epoch 857/1000
213/213 [==============================] - 0s 

213/213 [==============================] - 0s 147us/sample - loss: 0.0156 - recall: 0.9720
Epoch 925/1000
213/213 [==============================] - 0s 122us/sample - loss: 0.0156 - recall: 0.9720
Epoch 926/1000
213/213 [==============================] - 0s 181us/sample - loss: 0.0156 - recall: 0.9720
Epoch 927/1000
213/213 [==============================] - 0s 159us/sample - loss: 0.0156 - recall: 0.9720
Epoch 928/1000
213/213 [==============================] - 0s 185us/sample - loss: 0.0156 - recall: 0.9720
Epoch 929/1000
213/213 [==============================] - 0s 207us/sample - loss: 0.0156 - recall: 0.9720
Epoch 930/1000
213/213 [==============================] - 0s 113us/sample - loss: 0.0156 - recall: 0.9720
Epoch 931/1000
213/213 [==============================] - 0s 161us/sample - loss: 0.0156 - recall: 0.9720
Epoch 932/1000
213/213 [==============================] - 0s 128us/sample - loss: 0.0156 - recall: 0.9720
Epoch 933/1000
213/213 [==============================] - 0s 

In [72]:
from scipy.spatial.distance import euclidean

string_rep1 = string_featurization_model.predict(
    ngram_featurizer.transform(data['string1'])
)
string_rep2 = string_featurization_model.predict(
    ngram_featurizer.transform(data['string2'])
)
dists = np.zeros(shape=(len(data), 1))
for i, (v1, v2) in enumerate(zip(string_rep1, string_rep2)):
    dists[i] = euclidean(v1, v2)
    
roc_auc_score(data['matched'].astype(float), 1 - dists)
0.9802944806912361


0.9802944806912361

In [73]:
# some very basic plotting stuff
# maybe useful for training

import matplotlib.pyplot as plt
import matplotlib as mpl


def plot_metrics(history, metrics=['loss', 'auc', 'precision', 'recall']):
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        
    mpl.rcParams['figure.figsize'] = (12, 10)
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch,  history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
        plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
        plt.ylim([0.8,1])
    else:
        plt.ylim([0,1])

    plt.legend()


#plot_metrics(history, metrics=['recall'])

In [74]:
!pip install annoy  # https://github.com/spotify/annoy

In [84]:
# given featurized strings, we can use nearest neighbor-search/classification
# to decide very quickly which string matches and how well it matches.

from annoy import AnnoyIndex

index = AnnoyIndex(company_space.shape[1], 'euclidean')
for i, company in enumerate(company_list):
    emb = ngram_featurizer.transform([company]).toarray().flatten()
    index.add_item(i, emb)
    
index.build(10)
index.save('string_list.ann')

True

In [88]:
company_list

['Blackrock',
 'Credit Suisse',
 'Goldman Sachs',
 'Bank of America/Meryll Lynch',
 'Morgan Stanley',
 'LEK',
 'JP Morgan',
 'Nomura',
 'BNP Paribas',
 'WPP',
 'Rothschild',
 'Allianz']

In [89]:
#ngram_featurizer.get_feature_names()

In [94]:
v = ngram_featurizer.transform(['Allianz AG']).toarray().flatten()
#ngram_featurizer.transform(['HSBC']).toarray()
neighbours, distances = index.get_nns_by_vector(v, 3, search_k=-1, include_distances=True)

In [95]:
company_list[neighbours[0]]

'Allianz'